In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Osteoporosis/GSE56814'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression study of blood monocytes in pre- and postmenopausal females with low or high bone mineral density (HuEx-1_0-st-v2)"
!Series_summary	"Comparison of circulating monocytes from pre- and postmenopausal females with low or high bone mineral density (BMD). Circulating monocytes are progenitors of osteoclasts, and produce factors important to bone metabolism. Results provide insight into the role of monocytes in osteoporosis."
!Series_summary	"We identify osteoporosis genes by microarray analyses of monocytes in high vs. low hip BMD (bone mineral density) subjects."
!Series_overall_design	"Microarray analyses of monocytes were performed using Affymetrix 1.0 ST arrays in 73 Caucasian females (age: 47-56) with extremely high (mean ZBMD =1.38, n=42, 16 pre- and 26 postmenopausal subjects) or low hip BMD (mean ZBMD=-1.05, n=31, 15 pre- and 16 postmenopausal subjects). Differential gene expression analysis in high vs. low BMD subjects was cond

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

sample_characteristics = {
    0: ['gender: Female'], 
    1: ['bone mineral density: high BMD', 'bone mineral density: low BMD'], 
    2: ['state: postmenopausal', 'state: premenopausal'], 
    3: ['cell type: monocytes']
}

# Determining if gene expression data is available
is_gene_available = True  # Based on the background information provided

# Data Availability and Identifying the Keys
# Checking for Osteoporosis information
for key, values in sample_characteristics.items():
    if 'bone mineral density' in ''.join(values).lower():
        trait_row = key if len(set(values)) > 1 else None

# Checking for Age information
# Age is mentioned in the dataset background, but not in the sample characteristics dictionary directly
age_row = None

# Checking for Gender information
for key, values in sample_characteristics.items():
    if 'gender' in ''.join(values).lower():
        gender_row = key if len(set(values)) > 1 else None

# Data Type Conversion Functions
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if 'high' in value:
        return 1
    elif 'low' in value:
        return 0
    else:
        return None

def convert_age(value):
    value = value.split(':')[-1].strip().lower()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# Saving cohort information
save_cohort_info('GSE56814', './preprocessed/Osteoporosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Osteoporosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Osteoporosis/trait_data/GSE56814.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1369683': [1], 'GSM1369684': [1], 'GSM1369685': [1], 'GSM1369686': [1], 'GSM1369687': [0], 'GSM1369688': [0], 'GSM1369689': [1], 'GSM1369690': [1], 'GSM1369691': [1], 'GSM1369692': [1], 'GSM1369693': [0], 'GSM1369694': [1], 'GSM1369695': [0], 'GSM1369696': [1], 'GSM1369697': [1], 'GSM1369698': [0], 'GSM1369699': [0], 'GSM1369700': [0], 'GSM1369701': [0], 'GSM1369702': [1], 'GSM1369703': [0], 'GSM1369704': [0], 'GSM1369705': [0], 'GSM1369706': [0], 'GSM1369707': [1], 'GSM1369708': [1], 'GSM1369709': [1], 'GSM1369710': [1], 'GSM1369711': [1], 'GSM1369712': [1], 'GSM1369713': [1], 'GSM1369714': [1], 'GSM1369715': [0], 'GSM1369716': [1], 'GSM1369717': [0], 'GSM1369718': [0], 'GSM1369719': [0], 'GSM1369720': [1], 'GSM1369721': [1], 'GSM1369722': [0], 'GSM1369723': [0], 'GSM1369724': [1], 'GSM1369725': [0], 'GSM1369726': [1], 'GSM1369727': [1], 'GSM1369728': [1], 'GSM1369729': [1], 'GSM1369730': [1], 'GSM1369731': [1], 'GSM1369732': [1], 'GSM1369733': [0], 'GSM1369734': [1], 'GSM1369735

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for mapping
identifier_key = 'ID'  # same kind of identifiers as in STEP3
gene_symbol_key = 'gene_assignment'  # key storing the gene symbols

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)
